In [2]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, assemble
from qiskit.visualization import plot_bloch_multivector, plot_histogram, array_to_latex
from qiskit.compiler import transpile
from qiskit.providers.ibmq.job import job_monitor
from qiskit.result import marginal_counts

In [ ]:
def multiplier(int: num1, int: num2) -> int:
    """
    A function that returns the result of multiplication of num1 and num2.
    
    num1: integer positive value (the first parameter to the multiplier function)
    num2: integer positive value (the second parameter to the multiplier function)
    
    return: integer that represents num1 * num2
    """
    # First, we want to translate our num1 into qubit inputs.
    num1_qubits = np.floor(np.log2(num1)+1)
    num2_qubits = np.floor(np.log2(num2)+1)
    qubits1 = int(num1_qubits)
    qubits2 = int(num2_qubits)
    num1_bi = np.binary_repr(num1)
    num2_bi = np.binary_repr(num2)
    qc = QuantumCircuit(qubits1 + qubits2 + 2, 1)

    qc2 = QuantumCircuit(qubits2 + 1)
    
    qc.h(2* qubits1 + qubits2-1)
    
    # Now we use distribution rule to change the idea of multiplication into addition.
    for i in range(qubits1):
        if num1_bi[i] == "1":
            qc.x(i)
    for i in range(qubits2):
        if num2_bi[i] == "1":
            qc.x(i + int(num1_qubits))
    qc.x(qubits1+qubits2)
         
    for i in range(qubits2):
        qc2.crx(np.pi*2**(qubits2-i)/2**(qubits2),i, qubits2)
    
    u_gate = qc2.to_gate()
    cu_gate = u_gate.control(1)
    for i in range(qubits1):
        for k in range(qubits1-i):
            qc.append(cu_gate, [i] + [m+qubits1 for m in range(qubits2 + 1)])
    
   
    qc.measure(2* qubits1 + qubits2-1, 0)
    print(qc)
    print(qc2)
    svsim = Aer.get_backend('aer_simulator')
    result = transpile(qc, backend=svsim)
    result.save_statevector()
    qobj = assemble(result)
    final_state = svsim.run(qobj).result().get_statevector()
    return plot_bloch_multivector(final_state)